### Matrix Decomposition using gradient descent

In [10]:
# initialize P and Q, which will be R = P * Q.T

import numpy as np

R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])

num_users, num_items = R.shape
K=3

# initialize P and Q, using normally distributed randomization
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))
print("P:", P)
print("Q:", Q)

P: [[ 0.54144845 -0.2039188  -0.17605725]
 [-0.35765621  0.28846921 -0.76717957]
 [ 0.58160392 -0.25373563  0.10634637]
 [-0.08312346  0.48736931 -0.68671357]]
Q: [[-0.1074724  -0.12801812  0.37792315]
 [-0.36663042 -0.05747607 -0.29261947]
 [ 0.01407125  0.19427174 -0.36687306]
 [ 0.38157457  0.30053024  0.16749811]
 [ 0.30028532 -0.22790929 -0.04096341]]


In [11]:
P[[1,2,3], [1,1,1]]

array([ 0.28846921, -0.25373563,  0.48736931])

In [12]:
P[1,2]

-0.7671795656267608

### Cost function for R and P*Q.T, comparing only the nonzero values

In [13]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P, Q.T)
    
    # get nonzero indexes and compute rmse
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

### Use gradient descent to calculate P, Q

In [15]:
non_zeros = [(i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] >0]

steps=1000
learning_rate=0.01
r_lambda=0.01

for step in range(steps):
    for i, j, r in non_zeros:
        # get difference between actual and predicted value
        eij = r-np.dot(P[i,:], Q[j,:].T)
        # use SGD with regularization
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
        
    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0:
        print('### iteration step:', step, "RMSE:", rmse)
        

### iteration step: 0 RMSE: 3.2388050277987723
### iteration step: 50 RMSE: 0.4876723101369648
### iteration step: 100 RMSE: 0.1564340384819247
### iteration step: 150 RMSE: 0.07455141311978046
### iteration step: 200 RMSE: 0.04325226798579314
### iteration step: 250 RMSE: 0.029248328780878973
### iteration step: 300 RMSE: 0.022621116143829466
### iteration step: 350 RMSE: 0.019493636196525135
### iteration step: 400 RMSE: 0.018022719092132704
### iteration step: 450 RMSE: 0.01731968595344266
### iteration step: 500 RMSE: 0.016973657887570753
### iteration step: 550 RMSE: 0.016796804595895633
### iteration step: 600 RMSE: 0.01670132290188466
### iteration step: 650 RMSE: 0.01664473691247669
### iteration step: 700 RMSE: 0.016605910068210026
### iteration step: 750 RMSE: 0.016574200475705
### iteration step: 800 RMSE: 0.01654431582921597
### iteration step: 850 RMSE: 0.01651375177473524
### iteration step: 900 RMSE: 0.01648146573819501
### iteration step: 950 RMSE: 0.016447171683479155


In [16]:
pred_matrix = np.dot(P, Q.T)
print("predicted Matrix:\n", np.round(pred_matrix, 3))

predicted Matrix:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


### Using Matrix decomposition on collaborative filtering

In [17]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape
    # initialize P and Q, using normally distributed randomization
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
    
    break_count = 0
    
    non_zeros = [(i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] >0]
    
    for step in range(steps):
        for i, j, r in non_zeros:
            # get difference between actual and predicted value
            eij = r-np.dot(P[i,:], Q[j,:].T)
            # use SGD with regularization
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0:
            print('### iteration step:', step, "RMSE:", rmse)
            
    return P, Q
    
    

In [18]:
import pandas as pd
import numpy as np

movies = pd.read_csv('./ml-latest-small/movies.csv')
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

# get movie titles for the column
rating_movies = pd.merge(ratings, movies, on='movieId')
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

In [19]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step: 0 RMSE: 2.9023619751336867
### iteration step: 10 RMSE: 0.7335768591017927
### iteration step: 20 RMSE: 0.5115539026853442
### iteration step: 30 RMSE: 0.37261628282537446
### iteration step: 40 RMSE: 0.2960818299181014
### iteration step: 50 RMSE: 0.2520353192341642
### iteration step: 60 RMSE: 0.22487503275269854
### iteration step: 70 RMSE: 0.20685455302331537
### iteration step: 80 RMSE: 0.19413418783028685
### iteration step: 90 RMSE: 0.18470082002720403
### iteration step: 100 RMSE: 0.17742927527209104
### iteration step: 110 RMSE: 0.17165226964707486
### iteration step: 120 RMSE: 0.1669518194687172
### iteration step: 130 RMSE: 0.16305292191997542
### iteration step: 140 RMSE: 0.15976691929679643
### iteration step: 150 RMSE: 0.1569598699945732
### iteration step: 160 RMSE: 0.15453398186715428
### iteration step: 170 RMSE: 0.15241618551077643
### iteration step: 180 RMSE: 0.15055080739628307
### iteration step: 190 RMSE: 0.1488947091323209


In [20]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941


In [21]:
def get_unseen_movies(ratings_matrix, userId):
    # user_rating is a Series with movie title as index
    user_rating = ratings_matrix.loc[userId, :]
    
    # if value > 0, the movie is already seen
    already_seen = user_rating[user_rating > 0].index.tolist()
    
    movies_list = ratings_matrix.columns.tolist()
    unseen_list = [movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

In [22]:
def recomm_movie_by_userId(pred_df, userId, unseen_list, top_n=10):
    # get predicted ratings for unseen movies for a particular user
    # recommend movies according to highest predicted ratings
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [23]:
# get names of the movies that user havent yet seen
unseen_list = get_unseen_movies(ratings_matrix, 9)

# get latent factor based collaborative filtering recommendations
recomm_movies = recomm_movie_by_userId(ratings_pred_matrix, 9, unseen_list, top_n=10)

recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690
